<a href="https://colab.research.google.com/github/ThaoChie/DS-BDS_HN/blob/main/Real_Estate_HaNoi_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import time
from tqdm.notebook import tqdm

# Thư viện địa lý
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy.distance import geodesic

# Thư viện Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Cấu hình hiển thị
pd.set_option('display.max_columns', None)
sns.set(style="whitegrid")
import warnings
warnings.filterwarnings('ignore')

In [13]:
!pip install xgboost
!pip install geopy tqdm seaborn scikit-learn pandas matplotlib
!install jupyter


In [14]:
# --- 1. CÁC HÀM LÀM SẠCH DỮ LIỆU ---
def clean_price(price_str):
    """Chuyển đổi giá sang tỷ VNĐ"""
    if pd.isna(price_str): return np.nan
    try:
        # Xử lý chuỗi, bỏ 'triệu/m2', thay dấu phẩy thành chấm
        clean_str = str(price_str).lower().replace(' triệu/m²', '').replace(',', '.')
        clean_str = re.sub(r'[^\d.]', '', clean_str)
        return float(clean_str)
    except: return np.nan

def clean_area(area_str):
    """Chuyển đổi diện tích sang m2"""
    if pd.isna(area_str): return np.nan
    try:
        clean_str = str(area_str).lower().replace(' m²', '').replace(',', '.')
        return float(clean_str)
    except: return np.nan

def clean_bedroom(bed_str):
    """Lấy số phòng ngủ"""
    if pd.isna(bed_str): return np.nan
    try:
        clean_str = str(bed_str).lower().replace(' phòng', '').strip()
        if clean_str == 'nhiều hơn 10': return 11
        return float(clean_str)
    except: return np.nan

# --- 2. CẤU HÌNH GEOCODING (AN TOÀN) ---
HO_GUOM_COORDS = (21.0285, 105.8542) # Tọa độ Hồ Gươm

# Khởi tạo API (User Agent phải là duy nhất)
geolocator = Nominatim(user_agent="hanoi_housing_project_final_student_v3")

# Cấu hình RateLimiter (delay 1.5s để không bị ban IP)
geocode_service = RateLimiter(geolocator.geocode, min_delay_seconds=1.5)

def get_geo_info(address):

    try:
        query = f"{address}, Hà Nội, Việt Nam"

        location = geocode_service(query, timeout=10)

        if location:
            lat = location.latitude
            lon = location.longitude
            dist = geodesic((lat, lon), HO_GUOM_COORDS).km
            return lat, lon, dist
        else:
            return np.nan, np.nan, np.nan

    except (GeocoderTimedOut, GeocoderUnavailable):

        return np.nan, np.nan, np.nan
    except Exception:
        return np.nan, np.nan, np.nan

In [15]:
# 1. Load dữ liệu
file_path = '/content/VN_housing_dataset.csv'
df = pd.read_csv(file_path)

# 2. Áp dụng làm sạch
df['Price_per_m2'] = df['Giá/m2'].apply(clean_price)
df['Area'] = df['Diện tích'].apply(clean_area)
df['Bedroom'] = df['Số phòng ngủ'].apply(clean_bedroom)

# 3. Tính lại Tổng giá (Tỷ VNĐ) = (Giá/m2 * Diện tích) / 1000
df['Total_Price_Billion'] = (df['Price_per_m2'] * df['Area']) / 1000

# 4. Lọc dữ liệu nhiễu (Outliers) & Thiếu
df_clean = df.dropna(subset=['Total_Price_Billion', 'Area', 'Quận', 'Loại hình nhà ở'])

# Chỉ lấy nhà có diện tích 10-500m2, giá 0.5-100 tỷ
df_filtered = df_clean[
    (df_clean['Area'] >= 10) & (df_clean['Area'] <= 500) &
    (df_clean['Total_Price_Billion'] >= 0.5) & (df_clean['Total_Price_Billion'] <= 100)
].copy()

# Điền phòng ngủ thiếu bằng Median
df_filtered['Bedroom'] = df_filtered['Bedroom'].fillna(df_filtered['Bedroom'].median())

print(f"🔹 Dữ liệu gốc: {len(df)} dòng")
print(f"🔹 Dữ liệu sạch sẵn sàng xử lý: {len(df_filtered)} dòng")

🔹 Dữ liệu gốc: 82497 dòng
🔹 Dữ liệu sạch sẵn sàng xử lý: 81914 dòng


In [17]:
# --- CẤU HÌNH CHẠY ---
SAMPLE_SIZE = 2000    # Lấy 5000 dòng để demo
BATCH_SIZE = 500
SLEEP_TIME = 30

# 1. Lấy mẫu
if len(df_filtered) > SAMPLE_SIZE:
    df_run = df_filtered.sample(n=SAMPLE_SIZE, random_state=42).copy()
else:
    df_run = df_filtered.copy()

print(f"BẮT ĐẦU GEOCODING CHO {len(df_run)} DÒNG...")

lats, lons, dists = [], [], []
num_batches = int(np.ceil(len(df_run) / BATCH_SIZE))

for i in range(num_batches):
    start_idx = i * BATCH_SIZE
    end_idx = start_idx + BATCH_SIZE

    current_batch = df_run.iloc[start_idx:end_idx]
    print(f"\nĐang chạy Batch {i+1}/{num_batches} (Dòng {start_idx} -> {end_idx})...")

    for address in tqdm(current_batch['Địa chỉ'], leave=False):
        lat, lon, dist = get_geo_info(address)
        lats.append(lat)
        lons.append(lon)
        dists.append(dist)

    # --- LƯU TẠM (CHECKPOINT) ---

    temp_len = len(lats)
    df_run.iloc[:temp_len, df_run.columns.get_loc('Địa chỉ')+1] # Trick update

    df_temp = df_run.iloc[:temp_len].copy()
    df_temp['Lat'] = lats
    df_temp['Lon'] = lons
    df_temp['Dist_Center_Km'] = dists
    df_temp.to_csv('Hanoi_Housing_Geo_Temp.csv', index=False)

    if i < num_batches - 1:
        print(f"Nghỉ {SLEEP_TIME}s để API hồi phục...")
        time.sleep(SLEEP_TIME)

# Gán kết quả cuối cùng
df_run['Lat'] = lats
df_run['Lon'] = lons
df_run['Dist_Center_Km'] = dists

df_run.to_csv('Hanoi_Housing_Geo_Processed.csv', index=False)
print("HOÀN TẤT! ")

BẮT ĐẦU GEOCODING CHO 5000 DÒNG...

Đang chạy Batch 1/10 (Dòng 0 -> 500)...


  0%|          | 0/500 [00:00<?, ?it/s]

Nghỉ 30s để API hồi phục...

Đang chạy Batch 2/10 (Dòng 500 -> 1000)...


  0%|          | 0/500 [00:00<?, ?it/s]

Nghỉ 30s để API hồi phục...

Đang chạy Batch 3/10 (Dòng 1000 -> 1500)...


  0%|          | 0/500 [00:00<?, ?it/s]

Nghỉ 30s để API hồi phục...

Đang chạy Batch 4/10 (Dòng 1500 -> 2000)...


  0%|          | 0/500 [00:00<?, ?it/s]

Nghỉ 30s để API hồi phục...


KeyboardInterrupt: 

In [21]:
df_run = pd.read_csv('/content/Hanoi_Housing_Geo_Temp.csv')

df_moi = df_run.drop(columns=['Ngày', 'Dist_Center_Km'])

df_moi.to_csv('Hanoi_Housing_Geo_Processed.csv', index=False)

In [ ]:
df_run = pd.read_csv('/content/Hanoi_Housing_Geo_Processed.csv')


print(f"Số lượng dữ liệu cuối cùng để train model: {len(df_final)} dòng")

# 2. Vẽ bản đồ nhiệt giá nhà
plt.figure(figsize=(12, 8))
sns.scatterplot(
    data=df_final,
    x='Lon', y='Lat',
    hue='Total_Price_Billion',
    size='Total_Price_Billion',
    sizes=(20, 200),
    palette='coolwarm',
    alpha=0.7
)
plt.title('Bản đồ giá nhà Hà Nội (Heatmap)', fontsize=15)
plt.xlabel('Kinh độ (Longitude)')
plt.ylabel('Vĩ độ (Latitude)')
plt.legend(title='Giá (Tỷ)', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(False)
plt.show()

# 3. Xem tương quan giữa Khoảng cách trung tâm và Giá
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_final, x='Dist_Center_Km', y='Total_Price_Billion', alpha=0.5)
plt.title('Càng xa trung tâm giá càng giảm?')
plt.xlabel('Khoảng cách đến Hồ Gươm (km)')
plt.ylabel('Giá nhà (Tỷ)')
plt.show()

In [ ]:
# 1. Chọn Feature và Target
features = ['Area', 'Bedroom', 'Quận', 'Loại hình nhà ở', 'Lat', 'Lon', 'Dist_Center_Km']
target = 'Total_Price_Billion'

X = df_final[features]
y = df_final[target]

# 2. One-Hot Encoding (Biến đổi Quận/Loại hình thành số)
X = pd.get_dummies(X, columns=['Quận', 'Loại hình nhà ở'], drop_first=True)

# 3. Chia tập Train/Test (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Huấn luyện Random Forest
print("Đang huấn luyện mô hình...")
rf_model = RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)
print("Đã huấn luyện xong!")

In [ ]:
# 1. Đánh giá sai số
y_pred = rf_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("=== KẾT QUẢ ĐÁNH GIÁ ===")
print(f"Sai số trung bình (MAE): {mae:.3f} Tỷ VNĐ")
print(f"Sai số toàn phương (RMSE): {rmse:.3f} Tỷ VNĐ")

# 2. Biểu đồ mức độ quan trọng của các yếu tố (Feature Importance)
feature_imp = pd.Series(rf_model.feature_importances_, index=X_train.columns).sort_values(ascending=False).head(10)

plt.figure(figsize=(10, 5))
sns.barplot(x=feature_imp, y=feature_imp.index, palette='viridis')
plt.title("Top 10 Yếu tố ảnh hưởng nhất đến giá nhà")
plt.xlabel("Điểm quan trọng (Importance Score)")
plt.show()

# 3. Demo Dự báo giá nhà
def predict_price_demo(area, bedroom, address, quan, loai_hinh):
    # Lấy tọa độ
    lat, lon, dist = get_geo_info(address)
    if pd.isna(lat):
        print("Không tìm thấy địa chỉ này!")
        return

    # Tạo dataframe input
    input_data = pd.DataFrame({
        'Area': [area],
        'Bedroom': [bedroom],
        'Quận': [quan],
        'Loại hình nhà ở': [loai_hinh],
        'Lat': [lat],
        'Lon': [lon],
        'Dist_Center_Km': [dist]
    })

    # Xử lý one-hot encoding
    input_encoded = pd.get_dummies(input_data)
    input_encoded = input_encoded.reindex(columns=X_train.columns, fill_value=0)

    # Dự báo
    price = rf_model.predict(input_encoded)[0]
    print(f"--- KẾT QUẢ DỰ BÁO ---")
    print(f"Địa chỉ: {address}")
    print(f"Cách Hồ Gươm: {dist:.2f} km")
    print(f"Giá dự đoán: {price:.2f} Tỷ VNĐ")

# Thử nhập một căn nhà thực tế
predict_price_demo(
    area=50,
    bedroom=3,
    address="Đường Cầu Giấy, Hà Nội",
    quan="Quận Cầu Giấy",
    loai_hinh="Nhà ngõ, hẻm"
)